# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=5)

ERROR:ib_insync.wrapper:Error 321, reqId 1: Error validating request:-'aS' : cause - ALL account is not supported


<IB connected to 127.0.0.1:4002 clientId=5>

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in 'EURUSD USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2018, 6, 22, 5, 43, 15, 337546, tzinfo=datetime.timezone.utc), bid=1.16281, bidSize=1000000, ask=1.16283, askSize=1500000, prevBid=1.1628, prevBidSize=8500000, prevAsk=1.16282, prevAskSize=2500000, high=1.16385, low=1.16005, close=1.1603, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.16283

The following cell will start a 60 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]
contract2Row = {c: i for (i, c) in enumerate(contracts)}

def onPendingTickers(tickers):
    for t in tickers:
        row = contract2Row[t.contract]
        df.iloc[row, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.setCallback('pendingTickers', onPendingTickers)
ib.sleep(60)

,symbol,bidSize,bid,ask,askSize,high,low,close
0,EURUSD,8000000,1.16278,1.16281,500000,1.16385,1.16005,1.1603
1,USDJPY,2000000,109.964,109.966,3000000,110.075,109.85,109.97
2,GBPUSD,1000000,1.32718,1.32722,1000000,1.32825,1.32395,1.3245
3,USDCHF,4000000,0.9901,0.99017,4000000,0.99195,0.98995,0.99185
4,USDCAD,3000000,1.32969,1.32975,2000000,1.3322,1.32905,1.3316
5,AUDUSD,2500000,0.73959,0.73962,500000,0.7403,0.7374,0.73795


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

In [8]:
ib.disconnect()

***